In [1]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [2]:
# wandb.init(project="smode_rdata_reverse_label")

In [3]:
# # check if we have the correct number of checkpoint files 
# ppath ='/project/results/soutput/snapshots/' 
# for fold in range(15):
#     foldstr = f"fold{fold:0>2}"
#     cpath =os.path.join(ppath, foldstr)
#     files = os.listdir(cpath)
#     print(len(files), end=" ")

In [3]:
list(range(5))

[0, 1, 2, 3, 4]

In [4]:

args = argparse.Namespace()
args.gazeMpiimage_dir = '/project/data/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/Label'  #real label
args.output = '/project/results/soutput/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot='/project/results/soutput/snapshots/'
args.evalpath = '/project/results/sroutput/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=28
args.angle = 180
args.bin_width = 4


In [5]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width

In [6]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used=getArch(arch, bins)  #resnet50 and 28 bins

In [7]:
# fold=2
# folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
# folder.sort(key=natural_keys)
# folder.pop(-1)  #remove the tensorboard file
# # print(folder)
# epochs = folder[3]
# os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epochs)

In [8]:
# print(evalpath, snapshot_path)

In [9]:
# print(data_set)

In [10]:
%%time
all_MAE = []
# tosave={}
for fold in range(15):
    wandb.init(project="smode_rdata_reverse_label")
    print(f"fold={fold}")
    
    epoch_values=[]
    mae_values=[]

    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format)
    
    print(args.gazeMpiilabel_dir)
    folder = os.listdir(args.gazeMpiilabel_dir)
    folder.sort()  #individual label files
    testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder] 
    
    gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, args.gazeMpiimage_dir, transformations, False, angle, fold)

    test_loader = torch.utils.data.DataLoader(
        dataset=gaze_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True)

    fold_path = os.path.join(evalpath, 'fold' + f'{fold:0>2}'+'/')  #for evaluation
#     print(f"fold_path is {fold_path}")
    if not os.path.exists(fold_path):
        os.makedirs(fold_path)

#     if not os.path.exists(os.path.join(evalpath, f"fold"+str(fold))):
#         os.makedirs(os.path.join(evalpath, f"fold"+str(fold)))

    # list all epoch for testing
    folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
    folder.sort(key=natural_keys)
    folder.pop(-1)  #remove the tensorboard file, now all snapshot files
#     print(f"folder={folder}")
                    

    softmax = nn.Softmax(dim=1)
#     with open(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".log")), 'w') as outfile:
        
    configuration = (f"\ntest configuration equal gpu_id={gpu}, batch_size={batch_size}, model_arch={arch}\n"
                     f"Start testing dataset={data_set}, FOLD={fold} --{now}---------")
    print(configuration)
#     tosave['time':now]
#     outfile.write(configuration)
    epoch_list=[]
    avg_MAE=[]
    for epoch in folder: 
        print(f"entering epoch={epoch}")
        model=model_used
        checkpoint = torch.load(os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epoch))
        saved_state_dict = checkpoint['model_state_dict']
        model= nn.DataParallel(model,device_ids=[0])
        model.load_state_dict(saved_state_dict)
        model.cuda(gpu)
        model.eval()
        total = 0
        idx_tensor = [idx for idx in range(28)]
        idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 42
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 42

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
                    pl, yl = yl, pl
#                     yl = yl*(-1.0)
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl]))

        x = ''.join(filter(lambda i: i.isdigit(), epoch))
#             print(f"x={x}")
        
        epoch_list.append(x)
        mean_mae = avg_error/total  #mean mae over the 3000 iamges
        avg_MAE.append(mean_mae)  
#             print(f"total={total}")
        
        loger = f"[{epoch}---{args.dataset}] Total Num:{total},MAE:{mean_mae}"
#         outfile.write(loger)
        print(loger)
#             print(f"done epoch={epochs}")
        epochn = int(x)
#         wandb.log({'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold })
#         tosave ={'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold }
        epoch_values.append(epochn)
        mae_values.append(mean_mae)
# #         wandb.log(tosave)
        wandb.log({'epoch': epoch, f'fold_{fold}_mean_mae':mean_mae}, step=epochn)
       
    all_MAE.append(avg_MAE)
    wandb.finish()
#     data = [[x, y] for (x, y) in zip(epoch_values, mae_values)]
#     table = wandb.Table(data=data, columns = ["x", "y"])
#     wandb.log({"epoch_losses" : wandb.plot.line(table, "x", "y",
#                title="Epoch loss for each fold")})
    
#     fig = plt.figure(figsize=(14, 8))        
#     plt.xlabel('epoch')
#     plt.ylabel('avg')
#     plt.title('Gaze angular error')
#     plt.legend()
#     plt.plot(epoch_list, avg_MAE, color='k', label='mae')
#     fig.savefig(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".png")), format='png')
#     # plt.show() 

wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


fold=0
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=0 --06/20/2022 21:05:55---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:13.96576477015002
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:13.310476886626795
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:13.964950913253988
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:13.938010759117995
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:13.047713832480746
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:12.701151995302485
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:12.697705981177705
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:13.714712629553418
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] To

fold_0_mean_mae,█▄▇▃▁▆▄▇▁▄▃▆▅▅▄▃▆▂▃▃▅▇▄▅▄▃▄▅▃▄▃▆█▆▄▆▇▄▅▄
epoch,epoch_60.pkl
fold_0_mean_mae,13.35378


fold=1
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=1 --06/20/2022 21:20:20---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:12.313656857753333
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:11.901546817312619
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:11.141222875901208
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:10.968451740481804
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:10.783773419285248
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:10.868463089685612
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:10.667637538264225
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:14.180968262555458
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] T

fold_1_mean_mae,▄▃▂▁▁█▃▂▄▃▅▂▄▂▃▄▄▄▄▄▅▆▄▄▅▄▅▅▆▆▆█▆▆▇▅▇█▇▆
epoch,epoch_60.pkl
fold_1_mean_mae,13.11827


...

fold=13
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=13 --06/21/2022 00:13:26---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:11.10332914845438
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:12.027709468396369
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:12.055008106365518
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:12.472338344167351
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:11.14111707376403
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:11.406657086240395
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:11.470589459958413
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:12.357909917498924
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] T

fold_13_mean_mae,▁▅▆▁▂▆▇▅▃▅▅▄▆▃▅▄▅▅▄▆▅▆▄▅▄▃▅▄▅▄▄▅▅▆▃▅▆▆█▇
epoch,epoch_60.pkl
fold_13_mean_mae,12.6718


fold=14
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=14 --06/21/2022 00:27:52---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:15.788152016217147
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:14.3454567268855
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:15.76232974685266
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:15.864862304226396
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:16.60083878664961
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:17.393769591897794
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:15.359396749798583
entering epoch=epoch_8.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:16.744807709166757
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] Tot

fold_14_mean_mae,▄▁▄▅▃▆▆▅▆▅▆▅▇▆▆▆▆▆▇▇▇▇▇█▆▆▆▆▆▆▆▆▇▆▇▆▇█▇▇
epoch,epoch_60.pkl
fold_14_mean_mae,17.59057


CPU times: user 2h 54min 25s, sys: 1h 7min 35s, total: 4h 2min 1s
Wall time: 3h 36min 22s


In [11]:
all_MAE=np.array(all_MAE)

In [12]:

print(all_MAE.shape)
print(all_MAE.mean(axis=0))
print(all_MAE.mean(axis=1))
print(all_MAE.mean(axis=0).argmin()+1 ,all_MAE.mean(axis=0).min())

(15, 60)
[12.73320015 12.66976048 13.29805501 13.49525361 13.49816027 13.51472531
 13.30842204 14.09889858 14.05949045 13.85637639 13.90967756 14.32806288
 14.10930814 14.10788745 14.14207063 14.32835782 14.23040513 14.34655454
 14.26615159 14.00458119 14.25726836 14.20687374 14.35447629 14.36525256
 14.18271497 14.39796341 14.50344808 14.32886481 14.53107158 14.59748938
 14.40030267 14.69282299 14.43906777 14.34944433 14.60067985 14.61058843
 14.41827577 14.57232955 14.66060559 14.62001057 14.61284852 14.69078068
 14.71025778 14.62712429 14.61639218 14.69013644 14.78735022 14.63679245
 14.85162752 14.73501859 14.60493425 14.88924773 14.64028477 14.7535466
 14.90905922 14.91213516 14.91818184 14.808922   14.90649021 14.76218099]
[13.36636655 12.58951346 14.97902285 16.07620969 10.7099251  13.46431787
 18.8250614  15.43233654 13.56937345 12.01918444 16.02104838 12.47781844
 16.52140525 12.06828396 16.99469796]
2 12.669760482027044
